 Evaluate logistic and multivariate logistic models using ROC-AUC, precision, recall, and
 confusionmatrix.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    auc
)
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

#------------------------
# Binary Logistic Regression
#------------------------
# Filter for Setosa and Versicolor (classes 0 and 1)
binary_filter = y < 2
X_bin = X[binary_filter]
y_bin = y[binary_filter]


### Explanation for data loading and preprocessing (Cell below imports and prepares Iris dataset)

- import numpy as np: imports NumPy for numerical operations (arrays, math)
- import pandas as pd: imports pandas for DataFrame operations (not strictly used later but common practice)
- from sklearn.datasets import load_iris: imports a function to load the Iris dataset used for examples
- from sklearn.linear_model import LogisticRegression: imports the classifier used for both binary and multiclass logistic regression
- from sklearn.metrics import (...): imports metric functions for evaluation such as confusion_matrix, roc_auc_score, roc_curve and auc
  - classification_report: gives precision, recall, f1-score per class
  - confusion_matrix: tabulates true vs predicted class counts
  - roc_auc_score: numeric AUC score for ROC curve (works for binary and binarized multiclass)
  - roc_curve: computes false/true positive rates for different thresholds
  - auc: computes area under curve from fpr and tpr arrays
- from sklearn.preprocessing import label_binarize: used to convert multiclass labels into binary indicator matrix required for multiclass ROC-AUC
- import matplotlib.pyplot as plt: plotting library used for ROC curves
- import seaborn as sns: plotting themeing (not used but often helpful)

- iris = load_iris(): loads the dataset as a Bunch object with .data and .target
- X = iris.data: feature matrix (shape: samples x features)
- y = iris.target: integer labels (0,1,2)
- target_names = iris.target_names: string names for classes

- binary_filter = y < 2: selects only class 0 and 1 for a binary example
- X_bin = X[binary_filter], y_bin = y[binary_filter]: creates the binary subset used for the binary logistic model

Notes:
- We binarize the data for a separate binary logistic regression example to illustrate ROC and AUC, which are straightforward for binary classification. For multiclass ROC-AUC we will need to binarize labels differently (see later cell).
- Keeping imports explicit helps readability; unused imports (like pandas/seaborn) are harmless but can be removed if not needed.

In [ ]:
 # Train binary logistic model
model_bin = LogisticRegression()
model_bin.fit(X_bin, y_bin)
y_bin_pred = model_bin.predict(X_bin)
y_bin_prob = model_bin.predict_proba(X_bin)[:, 1]

 # Evaluation metrics for binary classification
print("=== Binary Logistic Regression ===")
print("Confusion Matrix:")
print(confusion_matrix(y_bin, y_bin_pred))
print("Classification Report:")
print(classification_report(y_bin, y_bin_pred, target_names= target_names[:2]))
print(f"ROC-AUC Score: {roc_auc_score(y_bin, y_bin_prob):.2f}")


### Explanation for binary logistic training and evaluation (Cell below trains the binary model)

- model_bin = LogisticRegression(): creates a LogisticRegression object with default parameters. Common params: penalty (regularization), C (inverse regularization strength), solver (optimiser). Defaults are fine for small examples.
- model_bin.fit(X_bin, y_bin): fits the logistic regression to the binary subset. Input shapes: X_bin (n_samples x n_features), y_bin (n_samples,)
- y_bin_pred = model_bin.predict(X_bin): returns predicted class labels (0 or 1) for each sample
- y_bin_prob = model_bin.predict_proba(X_bin)[:, 1]: returns predicted probabilities for the positive class (class 1). The output of predict_proba is shape (n_samples, n_classes); [:,1] selects column for class 1. For binary AUC we need a continuous score/probability, not just labels.

Evaluation lines:
- confusion_matrix(y_bin, y_bin_pred): compares true vs predicted labels to show counts of TP/FP/FN/TN
- classification_report(...): prints precision, recall, f1-score and support for each class; target_names maps numeric labels to readable names
- roc_auc_score(y_bin, y_bin_prob): computes AUC using true binary labels and predicted scores. AUC summarizes the ROC curve as a single number (1.0 perfect, 0.5 random).

Notes and why [:,1] but not a single index for multiclass:
- For binary classification predict_proba returns two columns (probabilities for class 0 and class 1). We select the positive-class column (class 1) as the score for ROC.
- For multiclass, predict_proba returns probabilities for each class; when computing multiclass ROC-AUC we either compute AUC per class using that class's probability column or binarize labels and pass the full probability matrix to roc_auc_score with an averaging method.

In [ ]:
 # Plot ROC Curve (Binary)
fpr, tpr, _ = roc_curve(y_bin, y_bin_prob)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f"ROC Curve (area = {auc(fpr, tpr):.2f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--')
plt.title('Binary Logistic Regression - ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("binary_roc_curve.png")
plt.show()


### Explanation for binary ROC plotting (Cell below creates the ROC plot)

- fpr, tpr, _ = roc_curve(y_bin, y_bin_prob): computes the Receiver Operating Characteristic curve points.
  - y_bin: true binary labels (0/1).
  - y_bin_prob: predicted scores (probability of positive class).
  - returns fpr (false positive rates), tpr (true positive rates), and thresholds (unused here, captured as _).
  - These arrays are used to plot TPR vs FPR across decision thresholds.

- plt.figure(figsize=(6,4)): creates a new figure with width 6in and height 4in.

- plt.plot(fpr, tpr, label=f"ROC Curve (area = {auc(fpr, tpr):.2f})", color='blue'): plots the ROC curve.
  - auc(fpr, tpr): computes area under curve from the two arrays.
  - label uses an f-string to display the AUC value in the legend.
  - color='blue' sets the curve color.

- plt.plot([0, 1], [0, 1], 'k--'): plots a diagonal baseline line from (0,0) to (1,1).
  - This diagonal represents a random classifier (TPR == FPR). If your ROC is above this line your model is better than random.
  - 'k--' is a matplotlib shorthand where 'k' = black color and '--' = dashed line.

- plt.title / plt.xlabel / plt.ylabel: set the plot title and axis labels.
- plt.legend(): shows the legend containing the AUC label.
- plt.grid(True): enables a grid to make it easier to read values.
- plt.tight_layout(): adjusts spacing to prevent overlap of labels/legend.
- plt.savefig('binary_roc_curve.png'): writes the figure to a PNG file.
- plt.show(): displays the figure inline in the notebook.

Notes and why ROC uses probabilities (scores) not labels:
- ROC curves show tradeoffs across thresholds. If you only pass predicted labels you get a single (FPR,TPR) point, so using predicted probabilities (a continuous score) lets you sweep thresholds and form a curve.
- auc(fpr,tpr) and roc_auc_score both summarize the ROC curve. roc_auc_score was used earlier for a numeric summary; auc(fpr,tpr) recomputes it from plotted arrays for display.

In [ ]:
 #------------------------
 # Multiclass Logistic Regression
 #------------------------
 # One-vs-Rest strategy (default in scikit-learn)
model_multi = LogisticRegression(multi_class='ovr', solver='liblinear')
model_multi.fit(X, y)
y_multi_pred = model_multi.predict(X)
y_multi_prob = model_multi.predict_proba(X)
 # Binarize y for ROC-AUC calculation
y_binarized = label_binarize(y, classes=[0, 1, 2])
 # Evaluation metrics for multiclass
print("\n=== Multiclass Logistic Regression ===")
print("Confusion Matrix:")
print(confusion_matrix(y, y_multi_pred))
print("Classification Report:")
print(classification_report(y, y_multi_pred, target_names=target_names))
print(f"Macro ROC-AUC Score: {roc_auc_score(y_binarized, y_multi_prob, average='macro'):.2f}")


### Explanation for multiclass training and ROC-AUC (Cell below trains multiclass model)

- model_multi = LogisticRegression(multi_class='ovr', solver='liblinear'): creates a logistic model configured for multiclass using One-vs-Rest (OvR) strategy.
  - multi_class='ovr': tells scikit-learn to train one binary classifier per class vs the rest. Alternative: 'multinomial' fits a single multinomial logistic regression.
  - solver='liblinear': an optimizer suitable for small datasets and OvR; some solvers required for certain penalties or for multinomial.

- model_multi.fit(X, y): fits the multiclass model on all three classes.
- y_multi_pred = model_multi.predict(X): predicts integer class labels (0,1,2).
- y_multi_prob = model_multi.predict_proba(X): returns an array shape (n_samples, n_classes) where each column is the predicted probability for that class.

- y_binarized = label_binarize(y, classes=[0,1,2]): converts the integer labels into a binary indicator matrix with shape (n_samples, n_classes).
  - Example: label 1 becomes [0,1,0]. This format is required to compute ROC curves/AUC per class because roc_curve expects binary labels for each class.

- print(confusion_matrix(...)) and classification_report(...): same roles as in binary case but they’ll show multiclass metrics.
- roc_auc_score(y_binarized, y_multi_prob, average='macro'): computes ROC-AUC by comparing the binarized true labels against the predicted probability matrix.
  - average='macro' computes AUC for each class and then averages them equally (unweighted by class support). Other options: 'weighted' (weighted by support), 'micro' (aggregate contributions of all classes).

Why we binarize y for multiclass ROC-AUC:
- ROC/AUC is defined for binary labels (true positive vs false positive for a given class). To extend to multiclass we compute one-vs-rest curves per class: binarize the true labels so each class becomes a separate binary problem.
- After binarization, we either compute per-class AUCs and average, or use methods that aggregate predictions.

Why there’s no [:,1] in the multiclass probability use here:
- In binary example we used [:,1] to select the positive-class probability for ROC. In multiclass we keep the full probability matrix (n_samples x n_classes) because roc_auc_score can accept the full score matrix together with a binarized y to compute per-class AUCs in one call.

Notes and practical tips:
- For small datasets scikit-learn defaults usually work; for larger/more complex problems you’ll tune 'C', choose different solvers, or use class_weight to handle imbalanced classes.
- When using 'ovr' you can still compute per-class ROC from y_multi_prob[:, i] vs y_binarized[:, i] if you prefer to plot them individually.

In [ ]:
 # Plot ROC curves for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_binarized[:, i], y_multi_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.figure(figsize=(6,4))
for i, color in zip(range(3), ['blue', 'green', 'red']):
    plt.plot(fpr[i], tpr[i], color=color,
              label=f"Class {target_names[i]} (AUC = {roc_auc[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
plt.title('Multiclass Logistic Regression - ROC Curves')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("multiclass_roc_curve.png")
plt.show()

### Explanation for multiclass ROC plotting (last plotting cell)

- fpr = {}, tpr = {}, roc_auc = {}: initialize dictionaries to store arrays and AUC values per class. Using dicts lets us index by class i (0,1,2).

- for i in range(3):
    - fpr[i], tpr[i], _ = roc_curve(y_binarized[:, i], y_multi_prob[:, i]): computes ROC curve for class i vs rest.
      - y_binarized[:, i] is the binary true-label vector for class i (1 where true class==i, 0 otherwise).
      - y_multi_prob[:, i] is the predicted probability for class i from predict_proba. We pass these two to get per-class ROC points.
    - roc_auc[i] = auc(fpr[i], tpr[i]): numeric area under the per-class ROC curve.
    - plt.figure(figsize=(6,4)): create a new figure for each class (so each class can be plotted on its own or combined depending on plotting approach).

- for i, color in zip(range(3), ['blue','green','red']):
    - plt.plot(fpr[i], tpr[i], color=color, label=...): draws the ROC curve for class i and adds the AUC value in the legend.
    - plt.plot([0,1],[0,1], 'k--'): draws the baseline diagonal indicating random classifier performance. The further above the curve is compared to this diagonal, the better the classifier.

- plt.title / plt.xlabel / plt.ylabel / plt.legend / plt.grid / plt.tight_layout / plt.savefig / plt.show: same roles as earlier (labels, legend, grid, layout, save and show the combined figure).

Why we compute per-class ROC and use dicts/arrays:
- ROC is inherently binary; for multiclass we compute one-vs-rest per class, so we need separate (fpr,tpr) arrays per class. Storing them in dicts by class index keeps code organized.
- y_multi_prob is shape (n_samples, n_classes); indexing [:, i] picks the score column for class i. This mirrors the binary case where [:,1] picked the positive-class probability.

Plotting choices and interpretation tips:
- You can either plot each class on the same figure (so curves overlap and are colored differently) or create separate figures per class; this code sets up per-class curves and plots them together for comparison.
- Use average='macro' or 'weighted' depending on whether you want equal weighting per class or weighting by class support when reporting a single scalar AUC.
- If classes are imbalanced prefer weighted averages or per-class reporting to see which classes the model struggles with.

Practical note on small dataset behavior:
- With only 50 samples per class (Iris dataset), ROC curves can look optimistic; always validate with cross-validation or a separate test set.

Summary:
- The plotting cell computes and visualizes per-class ROC curves using binarized true labels and predicted per-class probabilities. The diagonal baseline 'k--' shows random performance; AUC summarises curve quality.